In [16]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [17]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [18]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
}

folder_type = ['rgb','ndvi','thumb']

In [24]:

course_id = course_ids.get('포항')
course_id

'MGC001'

In [25]:
folders_candidate =glob.glob(os.path.join(out_folder,course_id,'*'))
folders_candidate

['D:\\ToAWS\\MGC001\\20230811', 'D:\\ToAWS\\MGC001\\20230816']

In [21]:
target_folder = folders_candidate[1]
target_folder

'D:\\ToAWS\\MGC001\\20230816'

In [8]:
target_date = os.path.split(target_folder)[-1]
target_date 

'20230816'

Gather all data json files from Output Folder

In [9]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['D:\\ToAWS\\MGC001\\20230816\\1.태백3H\\data202308161.태백3H.json',
 'D:\\ToAWS\\MGC001\\20230816\\2.태백5H\\data202308162.태백5H.json',
 'D:\\ToAWS\\MGC001\\20230816\\3.동해1H\\data202308163.동해1H.json',
 'D:\\ToAWS\\MGC001\\20230816\\4.동해2H\\data202308164.동해2H.json',
 'D:\\ToAWS\\MGC001\\20230816\\5.동해3H\\data202308165.동해3H.json',
 'D:\\ToAWS\\MGC001\\20230816\\6.태백1H\\data202308166.태백1H.json']

In [10]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r") as _json:
    dataJson= json.load(_json)
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [11]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(totalPhotoJson, final , ensure_ascii=False)

In [12]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [13]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)